## **Data ETL Pipeline using Python**

Import Libraries

In [1]:
import tensorflow.keras as keras
import numpy as np
import sqlite3
import pandas as pd


Transform Data

In [2]:
(xtrain, ytrain), (xtest, ytest) = keras.datasets.fashion_mnist.load_data()

print("Training data shape:", xtrain.shape)
print("Training labels shape:", ytrain.shape)
print("Test data shape:", xtest.shape)
print("Test labels shape:", ytest.shape)


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Training data shape: (60000, 28, 28)
Training labels shape: (60000,)
Test data shape: (10000, 28, 28)
Test labels shape: (10000,)


Load Data into SQLite Database

In [3]:
conn = sqlite3.connect('fashion_mnist.db')

conn.execute('''
CREATE TABLE IF NOT EXISTS images
(id INTEGER PRIMARY KEY AUTOINCREMENT,
image BLOB NOT NULL,
label INTEGER NOT NULL);
''')

for i in range(xtrain.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtrain[i].tobytes()), ytrain[i]])

conn.commit()

for i in range(xtest.shape[0]):
    conn.execute('INSERT INTO images (image, label) VALUES (?, ?)',
                [sqlite3.Binary(xtest[i].tobytes()), ytest[i]])

conn.commit()
conn.close()


Read Data from SQLite Database


In [4]:
conn = sqlite3.connect('fashion_mnist.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM images')
rows = cursor.fetchall()

data = pd.read_sql_query('SELECT * FROM images', conn)
conn.close()

print("Data loaded from database:")
print(data.head())


Data loaded from database:
   id                                              image    label
0   1  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...    b'\t'
1   2  b'\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00)\xb...  b'\x00'
2   3  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x16v\x1...  b'\x00'
3   4  b"\x00\x00\x00\x00\x00\x00\x00\x00!`\xaf\x9c@\...  b'\x03'
4   5  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00...  b'\x00'
